# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [46]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "gdrive/My Drive/fake_news_challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()


# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
train_bodies=pd.read_csv(project_path+"train_bodies.csv")
train_stances=pd.read_csv(project_path+"train_stances.csv")
dataset=pd.merge(train_bodies,train_stances)



<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [57]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [59]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [60]:
import numpy as np
import keras.utils
import time
from keras.callbacks import TensorBoard, CSVLogger
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model
from nltk.corpus import stopwords
import operator

Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t=Tokenizer(num_words=MAX_NB_WORDS,oov_token="<oov>")

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [69]:
t.fit_on_texts(dataset)
print(t.word_counts)
print(t.word_docs)
print(t.word_index)
print(t.document_count)

OrderedDict([('body', 1), ('id', 1), ('articlebody', 1), ('headline', 1), ('stance', 1)])
defaultdict(<class 'int'>, {'id': 1, 'body': 1, 'articlebody': 1, 'headline': 1, 'stance': 1})
{'<oov>': 1, 'body': 2, 'id': 3, 'articlebody': 4, 'headline': 5, 'stance': 6}
4


#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = dataset['articleBody'].values
from nltk.tokenize import sent_tokenize
articles = []
for i in range(texts.shape[0]):
  tokenised_line = sent_tokenize(texts[i])
  articles.append(tokenised_line)

## Check 2:

first element of texts and articles should be as given below. 

In [71]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [72]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [73]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np
data = np.zeros((texts.shape[0],MAX_SENTS , MAX_SENT_LENGTH ), dtype='int32')
# print(data[0][0][0])
articles =  np.asarray(articles)


tokenizer = Tokenizer(num_words=MAX_NB_WORDS )
tokenizer.fit_on_texts(texts)
print(tokenizer.word_index['overnight'])


for i, sentences in enumerate(articles):
  for j, line in enumerate(sentences):
    if j < MAX_SENTS:
      tokens = text_to_word_sequence(line)
      k=0
      for _, word in enumerate(tokens):        
        if k < MAX_SENT_LENGTH and tokenizer.word_index[word] < MAX_NB_WORDS:          
          data[i,j, k] = tokenizer.word_index[word]
          k = k + 1
      
      
word_index = tokenizer.word_index    
print(len(word_index))

2886
27427


### Check 3:

Accessing first element in data should give something like given below.

In [74]:
data[0, :, :]

array([[    3,   500,   532,  7117,    79,     3,  3664,   318,     5,
         4576,   369,     4,  1668,  2886,     1,    88,    12,   444,
            0,     0],
       [  727,    94,  1032,     3,  2963,  1778,     7,   171,     3,
         1223,  1095,  2062,   779,   154,     1,  2959,   471,     1,
          533,   227],
       [   88,  1033,  4027,  2275,    12,     3,  1052,  3220,    19,
            1,    88,     2,  1725,     1,   514,  1956,    15,     9,
            3,  3035],
       [  174,  3773,   980,   191,  2487,    42,  6689,  1665,  1210,
            5, 13146, 17648,     1,   751,    30,   712,  3901,    66,
           92,     0],
       [ 2275,    12,  2316,    38,  1058,   337,   816,     2,   359,
          255,  1796,     5,  4371,    67,   478,     0,     0,     0,
            0,     0],
       [    1,   779,   171,    19,     1,   532,    31,     3,  7312,
            4,  2099,  1210,     6,     3,  5172,     4,  1223,  1210,
           12,  3280],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:

texts_heading = dataset['Headline'].values
from nltk.tokenize import sent_tokenize

articles_heading = []
for i in range(texts_heading.shape[0]):
    tokenised_line = sent_tokenize(texts_heading[i])
    articles_heading.append(tokenised_line)
texts_heading[0]

from keras.preprocessing.text import text_to_word_sequence
import numpy as np

data_heading = np.zeros((texts_heading.shape[0], MAX_SENTS, MAX_SENT_LENGTH),
                        dtype='int32')

articles_heading = np.asarray(articles_heading)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts_heading)


for i, sentences in enumerate(articles_heading):
    for j, line in enumerate(sentences):
        if j < MAX_SENTS:
            tokens = text_to_word_sequence(line)
            k = 0
            for _, word in enumerate(tokens):
                if k < MAX_SENT_LENGTH and tokenizer.word_index[
                    word] < MAX_NB_WORDS:
                    data_heading[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
import pandas as pd

labels = dataset['Stance'].values
labels = pd.get_dummies(labels)
labels = np.asarray(labels )

In [78]:
labels

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [79]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [82]:
import numpy as np
count = int(np.floor(0.8*data.shape[0]))

x_train = data[0:count,:,:]
y_train = labels[0:count]

x_heading = data_heading[0:count,:,:]


print("x_train",x_train.shape,"y_train",y_train.shape,"x_heading",x_heading.shape)

x_train (39977, 20, 20) y_train (39977, 4) x_heading (39977, 20, 20)


In [83]:
import numpy as np
count = int(np.floor(0.2*data.shape[0]))

x_val = data[0:count,:,:]
y_val = labels[0:count]
x_heading_val = data_heading[0:count,:,:]

print("x_val",x_val.shape,"y_val",y_val.shape,"x_heading_val",x_heading_val.shape)

x_val (9994, 20, 20) y_val (9994, 4) x_heading_val (9994, 20, 20)


### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [84]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [86]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(word_index) + 1, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [87]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

# Concatenate articles and headings from training dataset
x_tr = np.matmul(x_train,x_heading)
print(x_tr.shape)

#  Concatenate articles and headings from validation dataset
x_te = np.matmul(x_val,x_heading_val)
print(x_te.shape)

(39977, 20, 20)
(9994, 20, 20)


### Model

In [88]:
vocab_size = len(word_index) + 1
embed_dim = 100
lstm_out = 400

# Build a Sequential model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=MAX_SENT_LENGTH*MAX_SENTS, trainable=False))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4,activation='softmax'))











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compile and fit the model

In [89]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['acc'])
print(model.summary())



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 100)          2742800   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 400, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 400)               801600    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 1604      
Total params: 3,546,004
Trainable params: 803,204
Non-trainable params: 2,742,800
_________________________________________________________________
None


In [90]:
model.fit(x_tr.reshape(x_tr.shape[0],400),y_train,epochs=10,batch_size=512,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
39977/39977 [==============================] - 57s 1ms/step - loss: 0.8627 - acc: 0.7228
Epoch 2/10
39977/39977 [==============================] - 55s 1ms/step - loss: 0.7999 - acc: 0.7304
Epoch 3/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.8000 - acc: 0.7304
Epoch 4/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.8027 - acc: 0.7302
Epoch 5/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.7990 - acc: 0.7303
Epoch 6/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.8000 - acc: 0.7304
Epoch 7/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.8005 - acc: 0.7304
Epoch 8/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.7991 - acc: 0.7304
Epoch 9/10
39977/39977 [==============================] - 56s 1ms/step - loss: 0.7985 - acc: 0.7304
Epoc

## Build the same model with attention layers included for better performance (Optional)

In [91]:
model.evaluate(x_te.reshape(x_te.shape[0],400), y_val, batch_size=32)

9994/9994 [==============================] - 82s 8ms/step


[0.7987634042680132, 0.7295377226455082]

In [0]:
import keras
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

# Build another model
model = Sequential()
model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=MAX_SENT_LENGTH*MAX_SENTS, trainable=False))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


## Fit the model and report the accuracy score for the model with attention layer (Optional)

In [93]:
model.fit(x_tr.reshape(x_tr.shape[0],400), y_train,validation_data=(x_te.reshape(x_te.shape[0],400), y_val), epochs = 15,  batch_size=50, verbose = 1)

Train on 39977 samples, validate on 9994 samples
Epoch 1/15
39977/39977 [==============================] - 6s 151us/step - loss: 0.8730 - acc: 0.7254 - val_loss: 0.7558 - val_acc: 0.7306
Epoch 2/15
39977/39977 [==============================] - 5s 136us/step - loss: 0.7636 - acc: 0.7332 - val_loss: 0.7062 - val_acc: 0.7451
Epoch 3/15
39977/39977 [==============================] - 5s 137us/step - loss: 0.7236 - acc: 0.7437 - val_loss: 0.6656 - val_acc: 0.7588
Epoch 4/15
39977/39977 [==============================] - 5s 137us/step - loss: 0.6862 - acc: 0.7557 - val_loss: 0.6311 - val_acc: 0.7788
Epoch 5/15
39977/39977 [==============================] - 6s 140us/step - loss: 0.6553 - acc: 0.7679 - val_loss: 0.6009 - val_acc: 0.7902
Epoch 6/15
39977/39977 [==============================] - 6s 139us/step - loss: 0.6311 - acc: 0.7771 - val_loss: 0.5844 - val_acc: 0.7954
Epoch 7/15
39977/39977 [==============================] - 6s 141us/step - loss: 0.6150 - acc: 0.7845 - val_loss: 0.5746 - v

In [94]:
model.evaluate(x_te.reshape(x_te.shape[0],400), y_val, batch_size=32)

9994/9994 [==============================] - 1s 57us/step


[0.5462695733413521, 0.8057834700939773]